In [1]:
# Importing necessary modules
import os
import urllib.request as request
import zipfile

In [2]:
# Changing the current directory to the parent directory
os.chdir("../")

# Getting the current working directory and printing it
%pwd

'c:\\Users\\P52s\\Documents\\01_python_projects\\ElasticNet_Model'

In [3]:
# Importing constants and utility functions from the WineX package
from WineX.constants import *
from WineX.utils.common import read_yaml, create_directories


In [4]:
from dataclasses import dataclass
from pathlib import Path

# Defining a data class for configuration related to data ingestion
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
# ConfigurationManager class for managing configurations
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        # Reading configuration, parameters, and schema from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Creating necessary directories
        create_directories([self.config.artifacts_root])

    # Method to get the data ingestion configuration
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Creating necessary directories
        create_directories([config.root_dir])

        # Creating a DataIngestionConfig object
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
# Importing logger and utility function from WineX package
from WineX import logger
from WineX.utils.common import get_size
from google.cloud import storage
import requests
import os


In [7]:
# Set your Google Cloud Storage credentials (replace with your actual credentials)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"

# Replace with your Google Cloud Storage bucket and file path
bucket_name = "wine_data_buckets"
file_path = "winequality-data.zip"
local_file_path = "artifacts/data_ingestion/winequality-data.zip"

In [8]:
# Class for handling data ingestion operations
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # Method to download the data file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # Downloading the file using urllib
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following info:\n{headers}")
        else:
            # Logging if the file already exists
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    # Method to download using gcp
    def download_from_gcs(self, bucket_name, file_path, local_file_path):
        client = storage.Client()
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(file_path)

        try:
            # Download the file to a local path
            blob.download_to_filename(local_file_path)
            logger.info(f"File downloaded from GCS to {local_file_path}")
        except Exception as e:
            logger.error(f"Error downloading file from GCS: {e}")



    # Method to extract the zip file
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        # Creating the unzip directory if it doesn't exist
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            # Extracting the contents of the zip file
            zip_ref.extractall(unzip_path)

In [9]:
# Exception handling for the main process
try:
    # Creating ConfigurationManager and obtaining data ingestion configuration
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    # Creating DataIngestion object and performing data download and extraction
    data_ingestion = DataIngestion(config=data_ingestion_config)
        # Download the file from Google Cloud Storage
    data_ingestion.download_from_gcs(bucket_name, file_path, local_file_path)
    data_ingestion.extract_zip_file()

except Exception as e:
    # Raising an exception if an error occurs
    raise e

[2023-12-09 18:12:32,332: INFO: common] yaml file: config\config.yaml loaded successfully
[2023-12-09 18:12:32,341: INFO: common] yaml file: params.yaml loaded successfully
[2023-12-09 18:12:32,362: INFO: common] yaml file: schema.yaml loaded successfully
[2023-12-09 18:12:32,381: INFO: common] created directory at: artifacts
[2023-12-09 18:12:32,389: INFO: common] created directory at: artifacts/data_ingestion


[2023-12-09 18:12:38,100: INFO: 2498216547] File downloaded from GCS to artifacts/data_ingestion/winequality-data.zip
